In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


# Start here

In [2]:
#Save checkpoint
# data.to_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint", header=True, index=False)

#Use if kernel crashes
data = pd.read_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint")
data.head()

,index,user,item,review,rating,real,date
0,0,5044,0,drink be bad the hot chocolate be water down a...,1.0,-1,2014-11-16
1,1,5045,0,this be the bad experience ive ever have a cas...,1.0,-1,2014-09-08
2,2,5046,0,this be locate on the site of the old spruce s...,3.0,-1,2013-10-06
3,3,5047,0,i enjoy coffee and breakfast twice at toast du...,5.0,-1,2014-11-30
4,4,5048,0,i love toast the food choice be fantastic i lo...,5.0,-1,2014-08-28


In [3]:
#Alternate approach

In [4]:
def binarize(n):
    if n==1:
        return 0
    else:
        return 1

data['real'] = data['real'].progress_apply(lambda n: binarize(n))
data.head()

data['review'] = data['review'].progress_apply(lambda st: str(st))

In [5]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data['review'], data['real'], test_size=0.20, random_state=42)

#Load this from now cause split is already available
X_train = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\X_train") 
X_train = X_train['review']
X_train = X_train.apply(lambda s: str(s))
X_test = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\X_test") 
X_test = X_test['review']
X_test = X_test.apply(lambda s: str(s))
y_train= pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\y_train", header=None) 
y_train = y_train[1]
y_test= pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\y_test", header=None) 
y_test = y_test[1]
X_train.head()

0    i fell madly in love with these tiny sandwich ...
1    my review will strictly be on what i order let...
2    im from ny and i can say this pizza be delicio...
3    i have never have coffee i enjoy in fact i thi...
4    the mediterranean version of chipotle decor be...
Name: review, dtype: object

In [6]:
tokenizer = Tokenizer(num_words= 20000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data1 = pad_sequences(sequences, maxlen=100)

In [7]:
inputs = Input(shape=[100])
layers = Embedding(20000,50,input_length=100)(inputs)
layers = LSTM(128, return_sequences=True)(layers)
layers = Dense(256)(layers)
layers = Dense(128)(layers)
layers = Activation('relu')(layers)
layers = Dropout(0.5)(layers)
layers = Dense(1)(layers)
layers = Activation('sigmoid')(layers)
model = Model(inputs=inputs,outputs=layers)

model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 50)           1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [8]:
model.fit(data1,y_train,batch_size=5000,epochs=10, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 389412 samples, validate on 97354 samples
Epoch 1/10
 35000/389412 [=>............................] - ETA: 10:14 - loss: 0.6933 - acc: 0.40 - ETA: 10:16 - loss: 0.6843 - acc: 0.63 - ETA: 10:21 - loss: 0.6680 - acc: 0.71 - ETA: 10:06 - loss: 0.6454 - acc: 0.75 - ETA: 9:52 - loss: 0.6340 - acc: 0.7742 - ETA: 9:36 - loss: 0.6652 - acc: 0.790 - ETA: 9:21 - loss: 0.6573 - acc: 0.8011

KeyboardInterrupt: 

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=100)

In [ ]:
# accr = model.evaluate(test_sequences_matrix,y_test)
y_pred = model.predict(test_sequences_matrix)

In [ ]:
tst = []
for i in y_test:
    if i == 1:
        tst.append(1)
    else:
        tst.append(0)

In [ ]:
attempt = []
for i in y_pred:
    if i[0] > 0.4:
        attempt.append(1)
    else:
        attempt.append(0)

In [ ]:
print(classification_report(tst, attempt))

In [84]:
res = pd.DataFrame(attempt)

In [87]:
res.to_csv(r"C:\Users\elonm\Desktop\res_nn.csv", header=None, index=False)